In [24]:
import os

from langchain_community.document_loaders import DirectoryLoader
from langchain_core.documents import Document

ignore_filenames = ("Test", "test")

source_path = os.path.join(os.getcwd(), "sources", "sanggi-wjg/spring-ai-geek")

loader = DirectoryLoader(path=source_path, glob="**/*.kt")
doc: Document
documents = [doc for doc in loader.load() if not any(ignore_filename in doc.metadata["source"] for ignore_filename in ignore_filenames)]

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.KOTLIN,
    chunk_size=1024,
    chunk_overlap=50,
)
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=50,
#     length_function=len,
#     separators=[""],
# )
documents = splitter.split_documents(documents)

In [35]:
from app.service import CodeChatService

vector_store = CodeChatService.get_vector_store("temp", drop_old=True)
ids = vector_store.add_documents(documents)

In [41]:
vector_store.similarity_search_with_relevance_scores("Controller", k=3, score_threshold=0.7)